# Hotdog vs not hotdog with image-classification and transfer learning

1. [Introduction](#Introduction)
1. [Understanding Transfer Learning](#Understanding-Transfer-Learning)
1. [Prerequisites](#Prerequisites)
1. [Prepare the data](#Prepare-the-data)
1. [Training the new Image Classification Model](#Training-the-new-Image-Classification-Model)
1. [Deployment of the model](#Deployment-of-the-model)
1. [Inference of test data](#Inference-of-test-data)
1. [Optimize the model based on the architecture](#Optimize-the-model-based-on-the-architecture)
1. [Clean up](#Clean-up)

## Introduction
This notebook will create a model to detect hotdogs. A SageMaker GroundTruth labeling job should have been run to create an Augmented Manifest file from a small subset of images. Unfortunately, that's not enough for training the model. Instead, the caltech256 dataset will be used to supplement this data. Once the data will be merged in the form of an Augmented Manifest, training will begin. Instead of starting from scratch which would require many more images to train on, the SageMaker built-in image-classification algorithm will be used with transfer learning to create the model. The model will then be deployed and tested against another small dataset. Finally, the model will be compiled using SageMaker Neo for a Raspberry Pi and a Jetson Nano.

---

## Understanding Transfer Learning
Before we move on, let’s look at Resnet, a NN topology that can achieve a high accuracy in image classification. It won the 2015 ImageNet Large Scale Visual Recognition Challenge for best object classifier, and it’s one of the most commonly used NNs for computer vision problems.

Topologies like Resnet are called a convolutional neural network (CNN) because the network’s input layers execute convolution operations on the input image. A convolution is a mathematical function that emulates the visual cortex of an animal. A CNN has several convolution layers that learn image filters. These filters extract features from the input images such as edges, parts, and bodies. These features are then routed through the hidden or inner layers to the output layer. In the context of image classification, the output layer has one output per category.



Consider a trained neural network capable of classifying a dog, as shown in the following image. 

![neuralnetexample](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2018/05/04/ImagesSageMaker5.png) 

The convolution layers extract some features from the dog image, and the rest of the layers route these features to the correct output of the last layer with a high confidence.

Transfer learning is a technique used for reducing the time required for training a new model. Instead of training your model from scratch, you can use a modified pre-trained model and continue training it with your dataset. That’s why it’s called transfer learning: the knowledge learned by one NN is transferring to another NN.

It means, for instance, that if you want a model that can classify cars brand/model/year and you already have a pre-trained model, it makes sense to change it a little bit and retrain it with your own dataset in order to create a new model that will solve this new problem.

The Amazon SageMaker built-in algorithm for image classification is already prepared for transfer learning. You just need to set a given parameter to true, and your model will use this technique.

---

## Prerequisites
This notebook assumes that you have ran a SageMaker GroundTruth labeling job and that it has completed.

Setting up the different variables to be used for training the model. 
* **role**: Will be the role ARN that SageMaker will use to create resources
* **sess**: Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
* **bucket**: Contains the name of the bucket that SageMaker will use. It will create it if it doesn't already exist. However, this should already exist as it's the bucket that was used for the SageMaker GroundTruth labeling job.
* **training_image**: The URI of the image-classification docker image stored in ECR

In [1]:
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri

# Get the role
role = sagemaker.get_execution_role()

# Set the SageMaker Client
sagemaker_client = boto3.client('sagemaker')

# Get the default bucket, create it if it doesn't exist
sess = sagemaker.Session()
bucket=sess.default_bucket()

# The prefix for everything in the bucket
prefix="hotdog"

# Get the URL of the image-classification docker image from ECR
training_image = get_image_uri(sess.boto_region_name, 'image-classification')
print(training_image)

825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:1


---

## Prepare the data
First, we will augment the SageMaker GroundTruth dataset with caltech256.

The caltech 256 dataset consist of images from 257 categories (the last one being a clutter category) and has 30k images with a minimum of 80 images and a maximum of about 800 images per category. 

As we only care about hotdogs and not hotdogs, we will take one image from each category as well as the entire clutter category to form the "nothotdog" category. We will also use the "108.hot-dog" category to supplement our current images.

The Amazon SageMaker Image Classification algorithm supports both RecordIO (application/x-recordio) and image (image/png, image/jpeg, and application/x-image) content types for training in file mode and supports RecordIO (application/x-recordio) content type for training in pipe mode. However you can also train in pipe mode using the image files (image/png, image/jpeg, and application/x-image), without creating RecordIO files, by using the augmented manifest format. 

With Pipe input mode, the dataset is streamed directly to the training instances instead of being downloaded first. This means that the training jobs start sooner, finish quicker, and need less disk space. Since the output of a SageMaker GroundTruth labeling job is an Augmented Manifest Image Format, we can directly use it for our training. Once downloaded, we will have to append the caltech 256 data to that augmented manifest.

The output of a SageMaker GroundTruth labeling job is an Augmented Manifest Image Format. So we will use that file as the basis and append to it the rest of the images from caltech256.

Then, we will split the data for training and validation into two augmented manifest files.

Finally, we will upload the images used from caltech256 and the two augmented manifest files to S3 to be ready for training.

### Download the caltech256 dataset and create the two categories
Two categories need to be created. The _hotdog_ and _nothotdog_. We will create 2 folders and copy the appropriate files into them.
* We will copy the _108.hot-dog_ category pictures into the _hotdog_ category folder.
* We will copy the _257.clutter_ category pictures into the _nothotdog_ category folder.
* We will copy one from from every category except the above two into the _nothotdog_ category folder.

In [2]:
#wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar -q
!aws s3 cp s3://jondion-public/256_ObjectCategories.tar .

download: s3://jondion-public/256_ObjectCategories.tar to ./256_ObjectCategories.tar


In [3]:
%%bash

# Remove old directories and create the new ones
rm -rf caltech256
rm -rf 256_ObjectCategories
mkdir -p caltech256/hotdog caltech256/nothotdog

# Extract images
tar -xf 256_ObjectCategories.tar

# Add the hotdogs to the hotdog dataset
cp 256_ObjectCategories/108.hot-dog/* caltech256/hotdog/
# Add all clutter to the nothotdog dataset
cp 256_ObjectCategories/257.clutter/* caltech256/nothotdog/

# Take one image from every category and move it to the nothotdog dataset
for i in 256_ObjectCategories/*; do
    for j in `ls $i/*.jpg | grep -v "108.hot-dog" | grep -v "257.clutter" | shuf | head -n 1`; do
        cp $j caltech256/nothotdog/
    done
done

### Merge the datasets
To merge the caltech256 dataset and the augmented output manifest from the SageMaker GroundTruth labeling job, we will first download the output manifest and load it as a JSON payload.

In [4]:
import json
groundTruthOutputManifest = sagemaker_client.list_labeling_jobs(MaxResults=1)['LabelingJobSummaryList'][0]['LabelingJobOutput']['OutputDatasetS3Uri']

!aws s3 cp $groundTruthOutputManifest output.manifest
!head output.manifest

# Import the file into output
with open('output.manifest', 'r') as f:
    manifest = [json.loads(line) for line in f.readlines()]

download: s3://sagemaker-us-east-2-761424745283/hotdog/hotdog10/manifests/output/output.manifest to ./output.manifest
{"source-ref":"s3://sagemaker-us-east-2-761424745283/hotdog/dataset/hotdog1.jpg","hotdog":1,"hotdog-metadata":{"confidence":0.55,"job-name":"labeling-job/hotdog10","class-name":"Hotdog","human-annotated":"yes","creation-date":"2019-11-06T00:37:57.829893","type":"groundtruth/image-classification"}}
{"source-ref":"s3://sagemaker-us-east-2-761424745283/hotdog/dataset/hotdog2.jpg","hotdog":1,"hotdog-metadata":{"confidence":0.55,"job-name":"labeling-job/hotdog10","class-name":"Hotdog","human-annotated":"yes","creation-date":"2019-11-06T00:37:57.829871","type":"groundtruth/image-classification"}}
{"source-ref":"s3://sagemaker-us-east-2-761424745283/hotdog/dataset/hotdog3.jpg","hotdog":1,"hotdog-metadata":{"confidence":0.57,"job-name":"labeling-job/hotdog10","class-name":"Hotdog","human-annotated":"yes","creation-date":"2019-11-06T00:36:51.015326","type":"groundtruth/image-cla

You can see that there are many fields that SageMaker GroundTruth automatically created. However, only the the _source-ref_ and _hotdog_ attributes are required for the training job. That's why we will only need to add those 2 attributes based on the caltech256 data.
* **source-ref**: The S3 location of the image
* **hotdog**: `0` means that there is no hotdog while `1` means that a hotdog was found

We will loop through each files in the category folder (hotdog and nothotdog) and append an entry in the augmented _manifest_. 

In [5]:
import os

hotdogdir = "caltech256/hotdog"
for fname in sorted(os.listdir(hotdogdir)):
    fpath = os.path.join(hotdogdir, fname)
    manifest.append({'source-ref': 's3://{}/{}/{}'.format(bucket, prefix, fpath),'hotdog': 1})

nothotdogdir = "caltech256/nothotdog"
for fname in sorted(os.listdir(nothotdogdir)):
    fpath = os.path.join(nothotdogdir, fname)
    manifest.append({'source-ref': 's3://{}/{}/{}'.format(bucket, prefix, fpath),'hotdog': 0})

### Split the dataset into training and validation
First, we will randomize the data and then split the data between training and validate at 80/20. The last step is to create the _train.manifest_ and _validation.manifest_ files with the appropriate data.

In [6]:
import numpy as np

# Use numpy to shuffle the data
np.random.shuffle(manifest)

# Split dataset to 80/20 ration for training and validation
dataset_size = len(manifest)
train_test_split_index = round(dataset_size*0.8)
train_data = manifest[:train_test_split_index]
validation_data = manifest[train_test_split_index:]

#Create training and validation files
num_training_samples = 0
with open('train.manifest', 'w') as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write('\n')
        num_training_samples += 1
    
with open('validation.manifest', 'w') as f:
    for line in validation_data:
        f.write(json.dumps(line))
        f.write('\n')

### Upload the data to S3
The last step is to upload the images used from caltech256 and both manifest files to S3.

In [7]:
# Upload the pictures to S3
s3train = 's3://{}/{}/caltech256/'.format(bucket, prefix)
!aws s3 cp caltech256 $s3train --recursive --quiet

# Upload the augmented manifest files to S3
s3train = 's3://{}/{}/train.manifest'.format(bucket, prefix)
!aws s3 cp train.manifest $s3train
s3validation = 's3://{}/{}/validation.manifest'.format(bucket, prefix)
!aws s3 cp validation.manifest $s3validation

upload: ./train.manifest to s3://sagemaker-us-east-2-761424745283/hotdog/train.manifest
upload: ./validation.manifest to s3://sagemaker-us-east-2-761424745283/hotdog/validation.manifest


---

## Training the new Image Classification Model
Now that we are done with all the setup that is needed, we are ready to train our image classification. To begin, we will create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.
### Training parameters
There are two kinds of parameters that need to be set for training. The first one are the parameters for the training job. These include:
* **training_image**: The URI to the docker image in ECR containing the algorithm.
* **role**: The role that the instances will use to training data in S3.
* **train_instance_count**: This is the number of instances on which to run the training. When the number of instances is greater than one, then the image classification algorithm will run in distributed settings. In Pipe mode, distributed training isn't supported.
* **train_instance_type**: This indicates the type of machine on which to run the training. For image classification, we support the following GPU instances for training: ml.p2.xlarge, ml.p2.8xlarge, ml.p2.16xlarge, ml.p3.2xlarge, ml.p3.8xlargeand ml.p3.16xlarge. We recommend using GPU instances with more memory for training with large batch sizes.
* **train_volume_size**: The size of the EBS volume attached to the training instance.
* **train_max_run**: Timeout in seconds for training. After this amount of time Amazon SageMaker terminates the job regardless of its current status.
* **input_mode**: The input mode that the algorithm supports.  Valid modes: _File_ - Amazon SageMaker copies the training dataset from the S3 location to a local directory. _Pipe_ - Amazon SageMaker streams data directly from S3 to the container via a Unix-named pipe. We will use Pipe here.
* **output_path**: This the s3 folder in which the training output is stored.
* **sagemaker_session**: Session object which manages interactions with Amazon SageMaker APIs and any other AWS services needed.

In [8]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
ic = sagemaker.estimator.Estimator(training_image,
                                     role, 
                                     train_instance_count=1, 
                                     train_instance_type='ml.p3.2xlarge',
                                     train_volume_size = 50,
                                     train_max_run = 86400,
                                     input_mode= 'Pipe',
                                     output_path=s3_output_location,
                                     sagemaker_session=sess)

Apart from the above set of parameters, there are hyperparameters that are specific to the algorithm. These are:

* **num_layers**: The number of layers (depth) for the network. We use 18 in this samples but other values such as 50, 152 can be used.
* **use_pretrained_model**: Set to 1 to use pretrained model for transfer learning.
* **image_shape**: The input image dimensions,'num_channels, height, width', for the network. It should be no larger than the actual image size. The number of channels should be same as the actual image.
* **num_classes**: This is the number of output classes for the new dataset. Imagenet was trained with 1000 output classes but the number of output classes can be changed for fine-tuning. We only have 2 for our example.
* **num_training_samples**: This is the total number of training samples. It is set to 15240 for caltech dataset with the current split.
* **mini_batch_size**: The number of training samples used for each mini batch. In distributed training, the number of training samples used per batch will be N * mini_batch_size where N is the number of hosts on which training is run.
* **epochs**: Number of training epochs.
* **learning_rate**: Learning rate for training.
* **top_k**: Report the top-k accuracy during training.
* **resize**: Resize the image before using it for training. The images are resized so that the shortest side is of this parameter. If the parameter is not set, then the training data is used as such without resizing.


In [9]:
ic.set_hyperparameters(num_layers = 18,
                         use_pretrained_model = 1,
                         image_shape = "3,224,224",
                         num_classes = 2,
                         num_training_samples = num_training_samples,
                         mini_batch_size = 32,
                         epochs = 30,
                         learning_rate = 0.001,
                         top_k = 2,
                         resize = 224)

### Channel configuration
We then create a channel configuration for S3 data sources that provide additional information as well as the path to the training dataset using a `sagemaker.session.s3_input`.
* **s3_data**: The S3 key to the augmented manifest file.
* **content_type**: MIME type of the input data.
* **s3_data_type**: When set to 'AugmentedManifestFile', then ``s3_data`` defines a single S3 augmented manifest file listing the S3 data to train on.
* **attribute_names**: A list of one or more attribute names to use that are found in a specified AugmentedManifestFile.

In [10]:
train_data = sagemaker.session.s3_input(s3_data=s3train, 
                                        content_type='application/x-recordio', 
                                        record_wrapping='RecordIO',
                                        s3_data_type='AugmentedManifestFile', 
                                        attribute_names=["source-ref","hotdog"])

validation_data = sagemaker.session.s3_input(s3_data=s3validation, 
                                             content_type='application/x-recordio', 
                                             record_wrapping='RecordIO',
                                             s3_data_type='AugmentedManifestFile', 
                                             attribute_names=["source-ref","hotdog"])

data_channels = {'train': train_data, 'validation': validation_data}

### Start the training
Start training by calling the fit method from the estimator passing the information about the training data via the _inputs_ argument.

In [11]:
ic.fit(inputs=data_channels)

2019-11-06 00:42:29 Starting - Starting the training job...
2019-11-06 00:42:30 Starting - Launching requested ML instances...
2019-11-06 00:43:24 Starting - Preparing the instances for training......
2019-11-06 00:44:20 Downloading - Downloading input data...
2019-11-06 00:44:37 Training - Downloading the training image....Docker entrypoint called with argument(s): train
[11/06/2019 00:45:35 INFO 140445251127104] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[11/06/2019 00:45:35 INFO 140445251127104] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'learning_rat

[11/06/2019 00:46:05 INFO 140445251127104] Epoch[7] Batch [20]#011Speed: 230.961 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[11/06/2019 00:46:05 INFO 140445251127104] Epoch[7] Train-accuracy=1.000000
[11/06/2019 00:46:05 INFO 140445251127104] Epoch[7] Train-top_k_accuracy_2=1.000000
[11/06/2019 00:46:05 INFO 140445251127104] Epoch[7] Time cost=3.160
[11/06/2019 00:46:06 INFO 140445251127104] Epoch[7] Validation-accuracy=0.980469
[11/06/2019 00:46:09 INFO 140445251127104] Epoch[8] Batch [20]#011Speed: 199.893 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[11/06/2019 00:46:10 INFO 140445251127104] Epoch[8] Train-accuracy=1.000000
[11/06/2019 00:46:10 INFO 140445251127104] Epoch[8] Train-top_k_accuracy_2=1.000000
[11/06/2019 00:46:10 INFO 140445251127104] Epoch[8] Time cost=3.594
[11/06/2019 00:46:10 INFO 140445251127104] Epoch[8] Validation-accuracy=0.980469
[11/06/2019 00:46:12 INFO 140445251127104] Epoch[9] Batch [20]#011Speed: 419.998 samples/sec#011

[11/06/2019 00:46:57 INFO 140445251127104] Epoch[26] Batch [20]#011Speed: 415.950 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[11/06/2019 00:46:57 INFO 140445251127104] Epoch[26] Train-accuracy=1.000000
[11/06/2019 00:46:57 INFO 140445251127104] Epoch[26] Train-top_k_accuracy_2=1.000000
[11/06/2019 00:46:57 INFO 140445251127104] Epoch[26] Time cost=1.919
[11/06/2019 00:46:57 INFO 140445251127104] Epoch[26] Validation-accuracy=0.980469
[11/06/2019 00:46:59 INFO 140445251127104] Epoch[27] Batch [20]#011Speed: 400.796 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[11/06/2019 00:47:00 INFO 140445251127104] Epoch[27] Train-accuracy=1.000000
[11/06/2019 00:47:00 INFO 140445251127104] Epoch[27] Train-top_k_accuracy_2=1.000000
[11/06/2019 00:47:00 INFO 140445251127104] Epoch[27] Time cost=1.987
[11/06/2019 00:47:00 INFO 140445251127104] Epoch[27] Validation-accuracy=0.980469
[11/06/2019 00:47:02 INFO 140445251127104] Epoch[28] Batch [20]#011Speed: 389.164 samp

---

## Deployment of the model

A trained model does nothing on its own. We now want to use the model to perform inference. We will deploy the created model by using the deploy method in the estimator. This will create an _Endpoint Configuration_ and an _Endpoint_.
* **endpoint_name**: Name to use for creating an Amazon SageMaker endpoint.
* **initial_instance_count**: Minimum number of EC2 instances to deploy to an endpoint for prediction.
* **instance_type**: Type of EC2 instance to deploy to an endpoint for prediction. We don't need a GPU in this case as we won't be constantly doing inference.

In [12]:
ic_classifier = ic.deploy(endpoint_name='hotdog',
                          initial_instance_count = 1,
                          instance_type = 'ml.m5.xlarge')

-------------------------------------------------------------------------------------------------!

---

## Inference of test data
The algorithm supports image/png, image/jpeg, and application/x-image for inference sent as a bytearray.

First, we will get 10 test images and store them in the _testdata_ folder.

In [13]:
!mkdir testdata
!aws s3 cp s3://jondion-public/hotdog_testdata/ testdata/ --recursive

download: s3://jondion-public/hotdog_testdata/hotdog3.jpg to testdata/hotdog3.jpg
download: s3://jondion-public/hotdog_testdata/hotdog5.jpg to testdata/hotdog5.jpg
download: s3://jondion-public/hotdog_testdata/hotdog1.jpg to testdata/hotdog1.jpg
download: s3://jondion-public/hotdog_testdata/nothotdog2.jpg to testdata/nothotdog2.jpg
download: s3://jondion-public/hotdog_testdata/nothotdog1.jpg to testdata/nothotdog1.jpg
download: s3://jondion-public/hotdog_testdata/nothotdog5.jpg to testdata/nothotdog5.jpg
download: s3://jondion-public/hotdog_testdata/nothotdog3.jpg to testdata/nothotdog3.jpg
download: s3://jondion-public/hotdog_testdata/hotdog4.jpg to testdata/hotdog4.jpg
download: s3://jondion-public/hotdog_testdata/hotdog2.jpg to testdata/hotdog2.jpg
download: s3://jondion-public/hotdog_testdata/nothotdog4.jpg to testdata/nothotdog4.jpg


Next, we will iterate over every files from the _testdata_ folder, convert it into a bytearray and use the `sagemaker.RealTimePredictor` _ic_classifier_ object to execute a prediction. Even though it looks like a function call, like many of the other function calls done to the SageMaker API, this is an API call using HTTPS POST to the SageMaker Endpoint we created above.

For each of the classes of our model (we only have 2: hotdog, nothotdog), it will return a probability. We will look at the highest probability of the two to determine if it's a hotdog or nothotdog.

In [14]:
# Define our 2 categories and the folder with the test pictures
object_categories = ['nothotdog', 'hotdog']
test_image_folder = 'testdata'

# Loop through all files
for fname in sorted(os.listdir(test_image_folder)):
    fpath = os.path.join(test_image_folder, fname)
    
    # Make sure it's a file
    if os.path.isfile(fpath):
        
        # Convert the file into a bytearray
        with open(fpath, 'rb') as f:
            payload = f.read()
            payload = bytearray(payload)

        # Predict
        ic_classifier.content_type = 'application/x-image'
        result = json.loads(ic_classifier.predict(payload))

        # The result will output the probabilities for all classes
        # Find the class with maximum probability and print the class index as well as it's probability
        index = np.argmax(result)
        print("The file " + fname + " is a " + object_categories[index] + " with a probability of " + str(result[index]))

The file hotdog1.jpg is a hotdog with a probability of 0.9970898032188416
The file hotdog2.jpg is a hotdog with a probability of 0.9174467325210571
The file hotdog3.jpg is a hotdog with a probability of 0.6538909077644348
The file hotdog4.jpg is a hotdog with a probability of 0.9830251932144165
The file hotdog5.jpg is a hotdog with a probability of 0.90871262550354
The file nothotdog1.jpg is a nothotdog with a probability of 0.9836713075637817
The file nothotdog2.jpg is a nothotdog with a probability of 0.996351957321167
The file nothotdog3.jpg is a nothotdog with a probability of 0.9999521970748901
The file nothotdog4.jpg is a nothotdog with a probability of 0.9969801306724548
The file nothotdog5.jpg is a nothotdog with a probability of 0.9981411695480347


---

## Optimize the model based on the architecture

We will use SageMaker Neo to train a machine learning model so it can be run at the edge. We could also have done the optimization to run it for our particular type of instance. In this case, we will create two models. One optimized for an NVIDIA Jetson Nano and one for a Raspberri Pi 3 B+. Amazon SageMaker Neo optimizes models to run up to twice as fast, with less than a tenth of the memory footprint, with no loss in accuracy.

We will use the `compile_model` function of the Estimator to create a new optimized model with the following parameters:
* **target_instance_family**: Identifies the device that you want to run your model after compilation. We will use jetson_nano and rasp3b
* **input_shape**: Specifies the name and shape of the expected inputs for your trained model in json dictionary form. [1, 3, 224, 224] means a single (1) RGB (3) image of size 224x224.
* **output_path**: Specifies the S3 location to store the compiled model.
* **framework**: The framework that is used to train the original model. The image-classification algorithm was trained under MXNET.
* **framework_version**: The version of the framework.

### NVIDIA Jetson Nano
You can ignore the warning in red which says that we can't deploy this model on SageMaker. This will be done via AWS IoT GreenGrass.

In [15]:
# jetson_model = ic.compile_model(target_instance_family = 'jetson_nano',
#                                  input_shape = {'data':[1, 3, 224, 224]},
#                                  output_path = 's3://{}/{}/neo_output/'.format(bucket, prefix),
#                                  framework = 'mxnet',
#                                  framework_version = '1.2.1')

### Raspberry Pi 3 B+
You can ignore the warning in red which says that we can't deploy this model on SageMaker. This will be done via AWS IoT GreenGrass.

In [16]:
# rasp3b_model = ic.compile_model(target_instance_family = 'rasp3b',
#                                  input_shape = {'data':[1, 3, 224, 224]},
#                                  output_path = 's3://{}/{}/neo_output/'.format(bucket, prefix),
#                                  framework = 'mxnet',
#                                  framework_version = '1.2.1')

The model created by the compilation includes the file _model-shapes.json_ which isn't used by the code used on the Raspberri Pi. In fact, it creates an issue when that file is included in the model. This should be fixed in a new version of the Deep Learning Runtime. 

To fix the issue, we will download the model, re-archive the model and upload it back to S3

In [17]:
# s3rasp3b = 's3://{}/{}/neo_output/model-rasp3b.tar.gz'.format(bucket, prefix)
# !aws s3 cp $s3rasp3b .
# !tar -zxf model-rasp3b.tar.gz --exclude model-shapes.json
# !tar -czf model-rasp3b.tar.gz compiled.params compiled_model.json compiled.so
# !aws s3 cp model-rasp3b.tar.gz $s3rasp3b
# !tar -ztf model-rasp3b.tar.gz
# !rm compiled.params compiled_model.json compiled.so model-rasp3b.tar.gz

---

## Clean up

When we're done with the endpoint, delete the Model and the Endpoint which will remove the Endpoint Configuration by default. The data will still be stored in S3 so make sure to remove it as well.

In [19]:
ic_classifier.delete_model()
ic_classifier.delete_endpoint()
print('Remove the data from the S3 bucket {} under the prefix {}.'.format(bucket, prefix))

Remove the data from the S3 bucket sagemaker-us-east-2-761424745283 under the prefix hotdog.
